# Assignment #4

Solve systems of linear equations using the following numerical methods:

1. Gauss Elimination
2. LU Factorization
3. Jacobi Method
4. Gauss-Seidel Iterative Method
Each method must be implemented in MATLAB/ Python.

Compare the methods based on the following criteria:
- Accuracy: Compare the final numerical solutions.
- Computational Efficiency: Measure the number of operations/iterations required.
- Stability: Discuss if the method is sensitive to rounding errors.
- Applicability: When is each method preferable?
(bonus) +++ Give an Example for Solving a real-world problem using linear equations.

In [1]:
from decimal import Decimal

import numpy as np
import pandas as pd
import sympy as sp
from IPython.display import display, Math


from utils.LaTeXTools import numpy_to_latex_gauss

In [2]:
# Linear equation in Matrix form
# columns are x_1, x_2,...,x_n, LHS
equations_augmented_matrix= sp.Matrix([
    [4,-1,2,11],
    [3,6,-1,8],
    [2,-1,5,7]
])
equations_coefficients_matrix= equations_augmented_matrix[:,:-1]
equations_lhs = equations_augmented_matrix[:,-1]
x_sym = sp.symbols(f'x_{{1:{equations_coefficients_matrix.shape[1]+1}}}')
x_sym_matrix = sp.Matrix(x_sym)
display(Math(
    f'{sp.latex(equations_coefficients_matrix)} '
    f'{sp.latex(x_sym_matrix)} = '
    f'{sp.latex(equations_lhs)}'
))

<IPython.core.display.Math object>

In [3]:
# Display Linear equations
sp.Eq(equations_coefficients_matrix * x_sym_matrix, equations_lhs)

Eq(Matrix([
[4*x_{1} - x_{2} + 2*x_{3}],
[3*x_{1} + 6*x_{2} - x_{3}],
[2*x_{1} - x_{2} + 5*x_{3}]]), Matrix([
[11],
[ 8],
[ 7]]))

## Gauss Elimination

In [4]:
gauss_elem_aug = equations_augmented_matrix.copy()
display(Math(numpy_to_latex_gauss(gauss_elem_aug)))

<IPython.core.display.Math object>

In [5]:
from SolveEquations.LinearEquations import gauss_naive, lu_decomposition_linear_solver

gauss_naive_solution = gauss_naive(
    a=equations_coefficients_matrix.copy(),
    b=equations_lhs.copy(),
    verbose=True
)

**Forward Elimination**

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**Back Substitution**

<IPython.core.display.Math object>

**Solution Vector**

<IPython.core.display.Math object>

In [6]:
gauss_naive_solution.evalf(10)

Matrix([
[ 2.582524272],
[0.1067961165],
[0.3883495146]])

## LU Decomposition

In [7]:
lu_decomposition_solution = lu_decomposition_linear_solver(
    a=equations_coefficients_matrix.copy(),
    b=equations_lhs.copy(),
    verbose=True
)

**LU Decomposition**

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**$Ly=b$ forward substitution**

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**$Lx=y$ back substitution**

<IPython.core.display.Math object>

**Solution Vector**

<IPython.core.display.Math object>

## LinearJacobiMethod

In [9]:
from SolveEquations.LinearJacobiMethod import LinearJacobiMethod

jacob_solver = LinearJacobiMethod(
    coefficients=equations_coefficients_matrix.copy(),
    lhs=equations_lhs.copy(),
    initial_guess=sp.zeros(equations_coefficients_matrix.shape[0],1),
)

jacob_solver_df = jacob_solver.run()

2025-03-23 14:38:54,064 - LinearJacobiMethod - INFO - Numerical.py:run:124 - Starting LinearJacobiMethod
2025-03-23 14:38:54,066 - LinearJacobiMethod - INFO - Numerical.py:initialize:70 - Initial state:{'x1': 0, 'x2': 0, 'x3': 0, 'residual': 15.2970585407784}
2025-03-23 14:38:54,068 - LinearJacobiMethod - INFO - Numerical.py:run:129 - Iteration 1 completed
Stop condition [StopIfEqual: Stop when 'residual' equals 0 (abs_tol=1e-06) for 3 iterations] NOT met: Variable residual:15.2971 != 0 (abs diff: 15.2971 > 1e-06)
2025-03-23 14:38:54,070 - LinearJacobiMethod - INFO - Numerical.py:run:134 - State: 
{'x1': 11/4, 'x2': 4/3, 'x3': 7/5, 'residual': 8.15074979509384}

2025-03-23 14:38:54,072 - LinearJacobiMethod - INFO - Numerical.py:run:129 - Iteration 2 completed
Stop condition [StopIfEqual: Stop when 'residual' equals 0 (abs_tol=1e-06) for 3 iterations] NOT met: Variable residual:8.15075 != 0 (abs diff: 8.15075 > 1e-06)
2025-03-23 14:38:54,073 - LinearJacobiMethod - INFO - Numerical.py:ru

In [12]:
jacob_solver_df.astype(float)

,x1,x2,x3,residual
0,0.000000,0.000000,0.000000,1.529706e+01
1,2.750000,1.333333,1.400000,8.150750e+00
2,2.383333,0.191667,0.566667,7.165698e-01
3,2.514583,0.236111,0.485000,5.627973e-01
4,2.566528,0.156875,0.441389,2.708812e-01
5,2.568524,0.123634,0.404764,6.930439e-02
6,2.578527,0.116532,0.397317,4.688575e-02
7,2.580474,0.110290,0.391896,1.583776e-02
8,2.581625,0.108412,0.389868,7.225221e-03
9,2.582169,0.107499,0.389033,3.267572e-03
